In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Retinoblastoma"
cohort = "GSE26805"

# Input paths
in_trait_dir = "../../input/GEO/Retinoblastoma"
in_cohort_dir = "../../input/GEO/Retinoblastoma/GSE26805"

# Output paths
out_data_file = "../../output/preprocess/Retinoblastoma/GSE26805.csv"
out_gene_data_file = "../../output/preprocess/Retinoblastoma/gene_data/GSE26805.csv"
out_clinical_data_file = "../../output/preprocess/Retinoblastoma/clinical_data/GSE26805.csv"
json_path = "../../output/preprocess/Retinoblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE26805_family.soft.gz', 'GSE26805_series_matrix.txt.gz']
Identified SOFT files: ['GSE26805_family.soft.gz']
Identified matrix files: ['GSE26805_series_matrix.txt.gz']

Background Information:
!Series_title	"Expression of p16 and Retinoblastoma Determines Response to CDK 4/6 Inhibition in Ovarian Cancer: Ovarian cancer cell line expression data."
!Series_summary	"PD-0332991 is a selective inhibitor of the CDK4/6 kinases with the ability to block retinoblastoma (Rb) phosphorylation in the low nanomolar range. Here we investigate the role of CDK4/6 inhibition in human ovarian cancer.  We examined the effects of PD-0332991 on proliferation, cell-cycle, apoptosis, and Rb phosphorylation using a panel of 40 established human ovarian cancer cell lines. Molecular markers for response prediction, including p16 and Rb, were studied using gene expression profiling, Western blot, and arrayCGH. Multiple drug effect analysis was used to study interactions with chem

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os

# Step 1: Assess gene expression data availability
# Based on background information, this appears to be gene expression data related to Retinoblastoma and p16
# expression in ovarian cancer cell lines
is_gene_available = True  # The dataset contains gene expression data

# Step 2: Identify variable availability and create conversion functions

# There's no direct information about which samples have retinoblastoma in the sample characteristics
# The dataset is about ovarian cancer cell lines, not retinoblastoma patients
# The trait "Retinoblastoma" refers to the Rb gene expression/status, not the disease itself
trait_row = None  # No direct classification of samples by Rb status in the characteristics

# No age information is provided for the cell lines
age_row = None

# No gender information is provided for the cell lines (cell lines don't have gender)
gender_row = None

# Define conversion functions (even though they won't be used in this case)
def convert_trait(value):
    # This would convert Rb status if it were available
    if not value or pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if value.lower() in ['positive', 'high', 'yes', '1']:
        return 1
    elif value.lower() in ['negative', 'low', 'no', '0']:
        return 0
    return None

def convert_age(value):
    if not value or pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    if not value or pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if value.lower() in ['female', 'f']:
        return 0
    elif value.lower() in ['male', 'm']:
        return 1
    return None

# Step 3: Save metadata
# Check if trait data is available
is_trait_available = trait_row is not None

# Save cohort info using the helper function
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Clinical Feature Extraction
# Since trait_row is None, we skip this step
# (No clinical data extraction is needed)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Based on our first step findings, we know there's only one file in the directory
# that matches the matrix file pattern
matrix_file = os.path.join(in_cohort_dir, "GSE26805_series_matrix.txt.gz")

# 2. Use the get_genetic_data function to extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file)
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['(+)E1A_r60_a104', '(+)E1A_r60_a97', '(+)E1A_r60_n9', '(+)eQC-41',
       'A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056',
       'A_23_P100074', 'A_23_P100092', 'A_23_P100103', 'A_23_P100111',
       'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156',
       'A_23_P100177', 'A_23_P100189', 'A_23_P100196', 'A_23_P100203'],
      dtype='object', name='ID')

Gene expression data shape: (41005, 40)


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, I can see they are Agilent microarray probe IDs (starting with "A_23_P")
# rather than standard human gene symbols (which would be like BRCA1, TP53, etc.)
# These microarray probe IDs need to be mapped to standard gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene identifiers and gene symbols
# From the preview, we can see:
# - The gene expression data uses 'ID' as identifiers (e.g., 'A_23_P100001')
# - The gene annotation data has 'ID' for the same identifiers and 'GENE_SYMBOL' for gene symbols

# 2. Extract the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# Print a preview of the mapping
print("Gene mapping preview:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the shape of the resulting gene expression data to verify the transformation
print(f"\nGene expression data after mapping - shape: {gene_data.shape}")
print("\nFirst 10 gene symbols after mapping:")
print(gene_data.index[:10])


Gene mapping preview:
             ID     Gene
0  A_23_P100001  FAM174B
1  A_23_P100011    AP3S2
2  A_23_P100022     SV2B
3  A_23_P100056   RBPMS2
4  A_23_P100074     AVEN

Gene expression data after mapping - shape: (18488, 40)

First 10 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAAS', 'AACS'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Since our dataset lacks clinical features (trait_row=None as determined in Step 2),
# we need a different approach for linking data
# Create a minimal clinical DataFrame with just sample IDs
sample_ids = normalized_gene_data.columns
clinical_features = pd.DataFrame(index=sample_ids)

# Add placeholder for trait column (all NaN)
clinical_features[trait] = float('nan')

# 3 & 4. Since we don't have trait data, we can't properly handle missing values
# or evaluate whether the trait is biased. Set appropriate flags.
is_trait_biased = True  # No trait data means we can't use this cohort for association studies
print("No trait data available for this cohort, marking as biased.")

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=False,  # We determined earlier that trait data is not available
    is_biased=is_trait_biased, 
    df=clinical_features,
    note="Dataset contains gene expression data from ovarian cancer cell lines but lacks Retinoblastoma classification information."
)

# 6. We've determined the data is not usable for association studies due to lack of trait information
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # This block likely won't execute but included for completeness
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # We don't have useful linked data to save
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset lacks trait information needed for association studies.")

Normalized gene data shape: (18247, 40)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC']


Normalized gene data saved to ../../output/preprocess/Retinoblastoma/gene_data/GSE26805.csv
No trait data available for this cohort, marking as biased.
Abnormality detected in the cohort: GSE26805. Preprocessing failed.
Data quality check result: Not usable
Data quality check failed. The dataset lacks trait information needed for association studies.
